# Parallel computation with Ray

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/coobas/europython-25/blob/main/04-ray.ipynb)

[Ray](https://docs.ray.io/en/latest/index.html) is a set of libraries that (among others) allow an easy parallelisation of Python tasks - both locally and also in clusters. This part is called **Ray Core**.

Apart from this, it also provides specialised libraries for data processing (**Ray Data**), for machine learning and even reinforcement learning (**Ray Train**, **Ray Train**, ...). We will not deal with those in this workshop.

In [ ]:
# Run this in Google Collab, not needed if you install this package locally
!pip install ray[default]

In [ ]:
# Obligatory imports
import numpy as np
import pandas as pd
import ray
import plotly.express as px
from IPython import get_ipython

from pathlib import Path

In [ ]:
is_colab = "google.colab" in str(get_ipython())

# Download the data which are part of this repo
if is_colab:
    import urllib
    url = "https://github.com/coobas/europython-25/raw/refs/heads/main/data.parquet"
    urllib.request.urlretrieve(url, "data.parquet")

In [ ]:
def long_running(i: int) -> int:
    """A long running task that we will parallelise."""
    import time
    time.sleep(1)
    return i * i

In [ ]:
%%time
long_running(1)

In [ ]:
%%time
[long_running(i) for i in range(10)]
    

If these tasks are independent, we can run them in parallel. There are of course options in Python itself:

- multiprocessing (https://docs.python.org/3.13/library/multiprocessing.html) 
- threading (with GIL-releasing code or with caution in free-threaded Python 3.13+)

This does scale exactly well if there are more tasks than CPUs / GPUs on your machine...

Other options:
- [celery](https://docs.celeryq.dev/en/stable/)
- [dask](https://docs.dask.org/en/stable/index.html)

With their strengths and weaknesses. 

## Use ray

Ray always runs a server (even implicitly) and executes the task in nodes that it manages.

In [ ]:
ray.init()

This either connects to an existing local server, or creates a new one. (In the same way, we can connect to a different one).

### Remote functions

Any function we decorate with the `ray.remote` decorator, becomes a **task** that can be submitted to this server.

In [ ]:
@ray.remote
def long_running_ray(i: int) -> int:
    import time
    time.sleep(1)
    return i * i

In [ ]:
long_running_ray(1)  # This will fail

Well, the error message is right. We should use `.remote` (a different one!)

Better (and see that we pass arguments the same way):

In [ ]:
%%time
long_running_ray.remote(1)

What happened? The task was submitted to the cluster. But asynchronously. We have to capture the **future** object...

In [ ]:
task_id = long_running_ray.remote(2)

...and get its value (synchronously):

In [ ]:
ray.get(task_id)

In [ ]:
%%time
task_ids = [long_running_ray.remote(i) for i in range(10)]
ray.get(task_ids)

## Exercise: Compute k nearest neighbours remotely

We will reuse our definitions of kNN functions (slightly modified) and for random points fínd their neighbours:

In [ ]:
DEFAULT_K = 4   # How many nearest neighbors to consider

def calculate_distances(query_points: np.ndarray, reference_points: np.ndarray, *, n_dim: int = 3) -> np.ndarray:
    """
    Calculate mutual Euclidean distances between M query and N reference points.

    Parameters:
    ----------
    query_points: np.ndarray
        (M, n_dim+) array of query points
    reference_points: np.ndarray
        (N, n_dim+) array of reference points
    n_dim: int
        Number of dimensions to consider (default: 3, for x, y, floor)

    Returns:
    --------
    distances: np.ndarray
        (M, N) array of the distances
    """
    # Expand for broadcasting
    query_points = query_points[:, np.newaxis,:n_dim]
    reference_points = reference_points[np.newaxis, :, :n_dim]
    return np.sqrt(np.sum((reference_points - query_points) ** 2, axis=-1))


def knn_search(
    query_points: np.ndarray,
    reference_points: np.ndarray,
    k: int,
):
    """
    Find k nearest neighbour reference point indices for N query points.

    Returns:
    --------
    indices: np.ndarray
        (N, k) matrix of integral indices
    """
    distances = calculate_distances(query_points, reference_points).T
    return np.argpartition(distances, k, axis=0)[:k].T

Let's create some points to test this on:

In [ ]:
def create_random_points(
    n_points: int, *, n_dim: int = 3, seed: int = 42
) -> np.ndarray:
    # TODO: Fix floor!
    np.random.seed(seed)
    return np.random.sample((n_points, n_dim))

In [ ]:
query_points = create_random_points(5, seed=42)
reference_points = create_random_points(10, seed=84)

query_points

In [ ]:
calculate_distances(query_points, reference_points)

First, we run this without ray (for sizable arrays):

In [ ]:
%%time
query_points = create_random_points(32768, seed=42)
reference_points = create_random_points(1000, seed=84)
knn_search(query_points, reference_points, k=DEFAULT_K)

**Task**

1. Change the knn_search function into a task.
2. Submit the computation to ray and compare the results.
3. Look at the execution times (and compare to local run)

In [ ]:
@ray.remote
def knn_search_ray(
    query_points: np.ndarray,
    reference_points: np.ndarray,
    k: int,
):
    """
    Find k nearest neighbour reference point indices for N query points.

    Returns:
    --------
    indices: np.ndarray
        (N, k) matrix of integral indices
    """
    distances = calculate_distances(query_points, reference_points).T
    return np.argpartition(distances, k, axis=0)[:k].T

In [ ]:
%%time
knn_id = knn_search_ray.remote(query_points, reference_points, k=DEFAULT_K)
knn_results = ray.get(knn_id)

**Question:** Is there any improvement yet?

## Monitoring ray

Ray comes with a nice dashboard that allows you to observe running jobs. It runs in a local web server, mostly likely http://localhost:8265. This address is not accessible when running within Google Colab, and so you have to use a special trick to show a mini-window forwarded to the dashboard running in the cloud.

In [ ]:
is_colab = "google.colab" in str(get_ipython())

if is_colab:
    from google.colab import output
    output.serve_kernel_port_as_iframe(8265)  # The port may differ!
else:
    print("Not in google Colab. Try the local link, it should work.")

Now we submit something really huge:

In [ ]:
query_points = create_random_points(10000000, seed=42)
reference_points = create_random_points(100, seed=84)
task_id = knn_search_ray.remote(query_points, reference_points, k=DEFAULT_K)


In [ ]:
%%time
ray.get(task_id)

## Exercise: Parallelize kNN execution

Ray by itself does not do any optimisations or segments the tasks you send it. So you need to care about splitting the task yourself. Sometimes this is easy, sometimes it requires a deeper thought (or a library ;-)).

If you want to improve the computation of nearest neighbours for a bunch of **independent** points, this is really easy ("embarassingly parallel problem). Just take the 1000000 query points and split them into **batches**:

In [ ]:
def split_into_batches(array: np.ndarray, max_size: int) -> list[np.ndarray]:
    """
    Split an array into smaller batches of a maximum size.

    Parameters:
    ----------
    array: np.ndarray
        The array to split.
    max_size: int
        The maximum size of each batch.

    Returns:
    --------
    list[np.ndarray]
        A list of arrays, each with a maximum size of `max_size`.
    """
    return [array[i:i + max_size] for i in range(0, len(array), max_size)]

In [ ]:
split_into_batches(np.arange(10), 3)

**Task**: 
1. Split the query points accordingly (you may experiment with batch_size, 1000 is pretty ok)
2. Submit and retrieve the results for the batches
3. Combine the results back into one array ([np.vstack](https://numpy.org/doc/stable/reference/generated/numpy.vstack.html) is your friend)

In [ ]:
query_points = create_random_points(10000000, seed=42)
reference_points = create_random_points(100, seed=84)

batches = split_into_batches(query_points, 1000)
batches

In [ ]:
%%time
task_ids = [
    knn_search_ray.remote(batch, reference_points, k=DEFAULT_K) for batch in batches
]
knn_results = ray.get(task_ids)
knn_results

In [ ]:
np.vstack(knn_results)

**Question** Can we parallelise in the reference points dimension?

## Example: Predicting the housing prices with a database of reference points

We can use the distances to predict the housing prices (in arbitrary units, perhaps the monthly rent in $ for a single-bedroom apartment?) based on our custom regression implementation. 

In [ ]:
def compute_prices(query_points, reference_points, k: int = DEFAULT_K):
    """
    Find prices for N data_points.

    Parameters:
    query_points: np.ndarray
        (N, 3) array of query points
    reference_points: np.ndarray
        (M, 4) array of data points with x, y, floor, and price
    k: int
        Number of nearest neighbors to consider

    Returns:
    --------
    prices: np.ndarray
        (N,) array of prices
    """
    indices = knn_search(query_points, reference_points, k)
    prices: np.ndarray = reference_points[indices, 3]
    return prices.mean(axis=1)


We have predefined reference points in an external file, modeled using an (unknown?) analytical function:

In [ ]:
is_colab = "google.colab" in str(get_ipython())

# Download the data which are part of this repo
if is_colab:
    import urllib
    url = "https://github.com/coobas/europython-25/raw/refs/heads/main/data.parquet"
    urllib.request.urlretrieve(url, "data.parquet")
    print("Data downloaded to data.parquet.")
else:
    print("Not in Google Colab, skipping data download.")

In [ ]:
def load_reference_points_df(path: Path = Path("../data.parquet")) -> np.ndarray:
    """
    Load reference data points from a parquet file.

    Returns:
    --------
    data_points: np.ndarray
        (N, 4) array of data points with x, y, floor, and price columns
    """

    return pd.read_parquet(path)
    # return df[["x", "y", "floor", "price"]].to_numpy().astype(float)

reference_points_df = load_reference_points_df()
reference_points_df

And we also want to run something against this. Let's start with random points:

In [ ]:
def create_random_3d_points(
        n_points: int = 100, *, ranges: list[tuple[float, float]] = [(-10, 10), (-10, 10), (1, 20)], ) -> np.ndarray:
    """
    Create random 3D points within specified ranges.

    Parameters:
    ----------
    n_points: int
        Number of points to generate.    
    ranges: list[tuple[float, float]]
        List of tuples specifying the range for each dimension.


    Returns:
    --------
    points: np.ndarray
        (n_points, len(ranges)) array of random points.
    """
    if not ranges or len(ranges) != 3:
        raise ValueError("Ranges must be a list of three tuples for x, y, and floor dimensions.")
    df = np.random.uniform(
        low=[r[0] for r in ranges],
        high=[r[1] for r in ranges],
        size=(n_points, len(ranges))
    )
    df[:, 2] = df[:, 2].astype(int)  # Ensure the floor is an integer
    return df

query_points = create_random_3d_points()
query_points

Our query points need to live in the same dimension (i.e. x,y between -10 and 10, floor between 1 and 20). Note that the floor distibution probably is not "uniform", but for demonstration purposes, it should not matter that much.

Let's see how the whole thing looks without ray:

In [ ]:
prices = compute_prices(query_points, reference_points_df.to_numpy(), k=DEFAULT_K)
prices

In [ ]:
def combine_points_and_prices(
    query_points: np.ndarray, prices: np.ndarray
) -> pd.DataFrame:
    """
    Prepare human-friendly output from numpy arrays.

    Parameters:
    ----------
    query_points: np.ndarray
        (N, 3) array of query points
    prices: np.ndarray
        (N,) array of prices

    Returns:
    --------
    df: pd.DataFrame
        DataFrame with columns x, y, floor, price
    """
    return pd.DataFrame(
        {
            "x": query_points[:,0],
            "y": query_points[:,1],
            "floor": query_points[:,2].astype(int),
        }
    )

In [ ]:
combine_points_and_prices(query_points, prices)

We already know how to parallelise this:

In [ ]:
# Note we can apply the decorator directly to the function
compute_prices_ray = ray.remote(compute_prices)

In [ ]:
%%time
query_points = create_random_3d_points(100_000)
batches = split_into_batches(query_points, 1_000)
task_ids = [
    compute_prices_ray.remote(batch, reference_points_df.to_numpy(), k=DEFAULT_K) 
    for batch in batches
]
np.hstack(ray.get(task_ids))

This is relatively fine but mind that we are passing the same reference points over and over again to ray (which requires repeated serialisation, ...). It does not matter that much in our case but if the reference points dataframe were larger, we might see a significant performance penalty.

### Storing objects in ray

What we can do instead, is to pass the object to ray just once and pass its **object id**. Ray automatically cares about using the referenced value.

Let's take it to the extreme and compute the price for each of 10,000 points in a separate task:

In [ ]:
%%time
query_points = create_random_3d_points(10_000)
batches = split_into_batches(query_points, 1_000)
task_ids = [
    compute_prices_ray.remote(batch, reference_points_df.to_numpy(), k=DEFAULT_K) 
    for batch in batches
]
ray.get(task_ids);

In [ ]:
%%time
query_points = create_random_3d_points(10_000)
batches = split_into_batches(query_points, 1)
task_ids = [
    compute_prices_ray.remote(batch, reference_points_df.to_numpy(), k=DEFAULT_K) 
    for batch in batches
]
ray.get(task_ids);

Do you see the degraded performance?

Of course it is mostly due to creating a task per point but part of the penalty is coming from passing the array so many times.

Putting objects into a ray cluster is very simple, actually:

In [ ]:
i = 9
arg_id = ray.put(i)
arg_id

In [ ]:
ray.get(long_running_ray.remote(arg_id))

**Task**

1. Modify the `compute_prices_ray` from above so that `reference_points_df.to_numpy()` is stored in the ray cluster and reused in each task.
2. Compare the execution time.

In [ ]:
%%time
query_points = create_random_3d_points(10000)
batches = split_into_batches(query_points, 1)
reference_id = ray.put(reference_points_df.to_numpy())
task_ids = [
    compute_prices_ray.remote(batch, reference_id, k=DEFAULT_K) 
    for batch in batches
]
ray.get(task_ids);

Is it better now?

**Question**: Would putting query points (either whole or batched) to ray help as well?

## TODO: Optional: Actors

We would like to create a map of prices, i.e. sample data in a grid over the allowed area and use our kNN model to predict a price for each of those (given the floor as a parameter). 

In [ ]:
def create_grid(n_points: int = N_POINTS) -> tuple[np.ndarray, ...]:
    """
    Create a homogenous grid of points to create a map.

    Returns:
    --------
    x: np.ndarray
        Flattened (n_points x n_points,) array of x values
    y: np.ndarray
        Flattened (n_points x n_points,) array of x values
    """
    # Note: Tested indirectly via `create_query_points`
    # TODO: Add floor
    x = np.linspace(-LIMIT, LIMIT, n_points)
    y = np.linspace(-LIMIT, LIMIT, n_points)
    return tuple(arr.flatten() for arr in np.meshgrid(x, y))


def create_query_points(n_points: int = N_POINTS, floor: int = 1) -> np.ndarray:
    """
    Create a homogenous grid of points with a floor to create a map.

    Returns:
    --------
    query_points: np.ndarray
        (n_points x n_points, 3) array of query points
    """
    x, y = create_grid(n_points=n_points)
    return np.vstack([x, y, np.ones(x.shape[0]) * floor]).T


create_query_points(2)

TODO: Create an actor?

In [ ]:
def draw_points(points: np.ndarray) -> None:
    """
    Draw points on a map.

    Parameters:
    -----------
    points: np.ndarray
        (N, 3) array of points to draw
    """
    df = pd.DataFrame({"x": points[:,0], "y": points[:,1]})
    fig = px.scatter(df, x="x", y="y", title="Query points")
    fig.show()

draw_points(create_query_points(5))